In [1]:
from utils.config import Config
from model.fasterrcnn import FasterRCNNTrainer, FasterRCNN
import tensorflow as tf
from utils.data import Dataset

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [2]:
config = Config()
config._parse({})

dataset = Dataset(config)
frcnn = FasterRCNN(21, (7, 7))

model = FasterRCNNTrainer(frcnn)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)



======user config========
{'max_size': 1000,
 'min_size': 600,
 'roi_sigma': 1.0,
 'rpn_sigma': 3.0,
 'voc_data_dir': 'D:/DataClinic/simple-faster-rcnn-pytorch/VOCdevkit/VOC2007/'}
==========end============


In [1]:
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [5]:
def predict(idx):
    img, bboxes, labels, scale = dataset[idx]
    img_size = img.shape[:2]
    x = tf.convert_to_tensor(img, dtype=tf.float32)
    x = tf.expand_dims(x, axis=0)
    
    bbox, label, score = model.faster_rcnn.predict(x, img_size)
    label = tf.cast(label, tf.int32).numpy()
    
    if len(label) == 0:
        return 'JINX'
    
    keep_idx = score > 0.8
    bbox = bbox[keep_idx]
    label = label[keep_idx]
    score = score[keep_idx]

    img = (img + 1) / 2.
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)


    for i in range(len(bbox)):
        y1 = bbox[i][0]
        x1 = bbox[i][1]
        y2 = bbox[i][2]
        x2 = bbox[i][3]
        height = y2 - y1
        width = x2 - x1
        ax.add_patch(plt.Rectangle((x1,y1), width, height, fill=False, edgecolor='red', linewidth=0.8))
        ax.text(x1,y1,VOC_BBOX_LABEL_NAMES[label[i]],style='italic',bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 0})
        ax.text(x2,y2,score[i],style='italic',bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 0})

In [ ]:
epoch = 5

for _ in range(epoch):
    for i in range(len(dataset)):
        img, bboxes, labels, scale = dataset[i]
        x = tf.convert_to_tensor(img, dtype=tf.float32)
        x = tf.expand_dims(x, axis=0)
        with tf.GradientTape() as tape:
            rpn_loc_l, rpn_cls_l, roi_loc_l, roi_cls_l = model(x, bboxes, labels, scale)
            total_loss = rpn_loc_l + rpn_cls_l + roi_loc_l + roi_cls_l
        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if i % 20 == 0:
            print("step", i)
            print("rpn_loc_loss = ", float(rpn_loc_l),
                  "rpn_cls_loss = ", float(rpn_cls_l),
                  "roi_loc_loss = ", float(roi_loc_l),
                  "roi_cls_loss = ", float(roi_cls_l))
        if i % 1000 == 0:
            model.save_weights('frcnn.h5')
            
        
        if i % 500 == 0:
            predict(1005)

W0626 19:22:08.476382  4972 deprecation.py:323] From d:\python\lib\site-packages\tensorflow_core\python\ops\array_grad.py:563: _EagerTensorBase.cpu (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.identity instead.


step 0
rpn_loc_loss =  0.10657259821891785 rpn_cls_loss =  0.691439151763916 roi_loc_loss =  0.018595486879348755 roi_cls_loss =  3.0464272499084473
step 20
rpn_loc_loss =  0.008573314175009727 rpn_cls_loss =  0.6538516879081726 roi_loc_loss =  0.045445311814546585 roi_cls_loss =  1.7432668209075928
step 40
rpn_loc_loss =  0.017442429438233376 rpn_cls_loss =  0.6793794631958008 roi_loc_loss =  0.02920326218008995 roi_cls_loss =  1.2720791101455688
step 60
rpn_loc_loss =  0.25246089696884155 rpn_cls_loss =  0.6841936111450195 roi_loc_loss =  0.0442499965429306 roi_cls_loss =  1.3727576732635498
step 80
rpn_loc_loss =  0.003468289040029049 rpn_cls_loss =  0.6795210838317871 roi_loc_loss =  0.035294126719236374 roi_cls_loss =  0.7979753017425537
step 100
rpn_loc_loss =  0.01193266175687313 rpn_cls_loss =  0.6790893077850342 roi_loc_loss =  0.0054526557214558125 roi_cls_loss =  0.6698393821716309
step 120
rpn_loc_loss =  0.008576503023505211 rpn_cls_loss =  0.6845340132713318 roi_loc_loss 

step 1080
rpn_loc_loss =  0.027359478175640106 rpn_cls_loss =  0.30744078755378723 roi_loc_loss =  0.0792740061879158 roi_cls_loss =  1.132646083831787
step 1100
rpn_loc_loss =  0.037596628069877625 rpn_cls_loss =  0.19988933205604553 roi_loc_loss =  0.024149268865585327 roi_cls_loss =  0.9891401529312134
step 1120
rpn_loc_loss =  0.03106028586626053 rpn_cls_loss =  0.23474831879138947 roi_loc_loss =  0.022141829133033752 roi_cls_loss =  0.8950393199920654
step 1140
rpn_loc_loss =  1.419292688369751 rpn_cls_loss =  1.0821901559829712 roi_loc_loss =  0.03223691135644913 roi_cls_loss =  1.289222002029419
step 1160
rpn_loc_loss =  0.16358447074890137 rpn_cls_loss =  0.6245754957199097 roi_loc_loss =  0.023759715259075165 roi_cls_loss =  0.9379816651344299
step 1180
rpn_loc_loss =  0.032431647181510925 rpn_cls_loss =  0.2229684740304947 roi_loc_loss =  0.02183702401816845 roi_cls_loss =  1.0085372924804688
step 1200
rpn_loc_loss =  0.058234378695487976 rpn_cls_loss =  0.28818538784980774 r

step 2160
rpn_loc_loss =  0.041154079139232635 rpn_cls_loss =  0.29890909790992737 roi_loc_loss =  0.042977653443813324 roi_cls_loss =  1.062312364578247
step 2180
rpn_loc_loss =  0.00397943239659071 rpn_cls_loss =  0.19555895030498505 roi_loc_loss =  0.03933078050613403 roi_cls_loss =  1.0133532285690308
step 2200
rpn_loc_loss =  0.11871005594730377 rpn_cls_loss =  0.1586945801973343 roi_loc_loss =  0.04413561150431633 roi_cls_loss =  1.1242787837982178
step 2220
rpn_loc_loss =  0.21080470085144043 rpn_cls_loss =  0.6691911220550537 roi_loc_loss =  0.048893943428993225 roi_cls_loss =  1.3353883028030396
step 2240
rpn_loc_loss =  0.11161531507968903 rpn_cls_loss =  0.4427039921283722 roi_loc_loss =  0.008834363892674446 roi_cls_loss =  0.7006251811981201
step 2260
rpn_loc_loss =  0.10938969254493713 rpn_cls_loss =  0.18532893061637878 roi_loc_loss =  0.05975776165723801 roi_cls_loss =  0.9722651243209839
step 2280
rpn_loc_loss =  0.04801985248923302 rpn_cls_loss =  0.3236284852027893 r

step 3240
rpn_loc_loss =  0.014811862260103226 rpn_cls_loss =  0.15987250208854675 roi_loc_loss =  0.04310477152466774 roi_cls_loss =  1.0661370754241943
step 3260
rpn_loc_loss =  0.008102286607027054 rpn_cls_loss =  0.17127470672130585 roi_loc_loss =  0.012945134192705154 roi_cls_loss =  0.4220436215400696
step 3280
rpn_loc_loss =  0.07940775156021118 rpn_cls_loss =  0.3330309987068176 roi_loc_loss =  0.0001737015845719725 roi_cls_loss =  0.23007655143737793
step 3300
rpn_loc_loss =  0.4112289249897003 rpn_cls_loss =  1.0163676738739014 roi_loc_loss =  0.014575527980923653 roi_cls_loss =  1.0347729921340942
step 3320
rpn_loc_loss =  0.01195688545703888 rpn_cls_loss =  0.15929606556892395 roi_loc_loss =  0.00838300958275795 roi_cls_loss =  0.7495545148849487
step 3340
rpn_loc_loss =  0.02764892578125 rpn_cls_loss =  0.26962044835090637 roi_loc_loss =  0.01415152009576559 roi_cls_loss =  0.6926705837249756
step 3360
rpn_loc_loss =  0.027246184647083282 rpn_cls_loss =  0.2955954670906067

...

step 2920
rpn_loc_loss =  0.06011282652616501 rpn_cls_loss =  0.21624822914600372 roi_loc_loss =  0.00849622581154108 roi_cls_loss =  0.629260778427124
step 2940
rpn_loc_loss =  0.03199499100446701 rpn_cls_loss =  0.12129692733287811 roi_loc_loss =  0.0298247542232275 roi_cls_loss =  1.0450741052627563
step 2960
rpn_loc_loss =  0.016584934666752815 rpn_cls_loss =  0.19361275434494019 roi_loc_loss =  0.02351408451795578 roi_cls_loss =  0.9787030220031738
step 2980
rpn_loc_loss =  0.03689569979906082 rpn_cls_loss =  0.19617803394794464 roi_loc_loss =  0.02102435752749443 roi_cls_loss =  1.2634834051132202
step 3000
rpn_loc_loss =  0.005716474261134863 rpn_cls_loss =  0.1886490434408188 roi_loc_loss =  0.014347639866173267 roi_cls_loss =  0.5523527264595032
step 3020
rpn_loc_loss =  0.028514519333839417 rpn_cls_loss =  0.33193540573120117 roi_loc_loss =  0.019945800304412842 roi_cls_loss =  0.6836894154548645
step 3040
rpn_loc_loss =  0.09793046116828918 rpn_cls_loss =  0.4113093912601471

step 4000
rpn_loc_loss =  0.31324684619903564 rpn_cls_loss =  0.9796423316001892 roi_loc_loss =  0.022196214646100998 roi_cls_loss =  1.1740919351577759
step 4020
rpn_loc_loss =  0.9044390916824341 rpn_cls_loss =  1.0372486114501953 roi_loc_loss =  0.004821513779461384 roi_cls_loss =  0.4122350811958313
step 4040
rpn_loc_loss =  0.06181502714753151 rpn_cls_loss =  0.22426240146160126 roi_loc_loss =  0.0071361493319272995 roi_cls_loss =  0.525099515914917
step 4060
rpn_loc_loss =  0.06036953628063202 rpn_cls_loss =  0.3606688380241394 roi_loc_loss =  0.011955062858760357 roi_cls_loss =  0.8956079483032227
step 4080
rpn_loc_loss =  0.0019241636618971825 rpn_cls_loss =  0.12490189075469971 roi_loc_loss =  0.007294537499547005 roi_cls_loss =  0.6929453015327454
step 4100
rpn_loc_loss =  0.08034572005271912 rpn_cls_loss =  0.31051281094551086 roi_loc_loss =  0.011318914592266083 roi_cls_loss =  0.8214067816734314
step 4120
rpn_loc_loss =  0.025385960936546326 rpn_cls_loss =  0.2685489058494

step 60
rpn_loc_loss =  0.2648462951183319 rpn_cls_loss =  0.492320716381073 roi_loc_loss =  0.03252103924751282 roi_cls_loss =  1.017827033996582
step 80
rpn_loc_loss =  0.00754902046173811 rpn_cls_loss =  0.2779189944267273 roi_loc_loss =  0.008072298020124435 roi_cls_loss =  0.5821017026901245
step 100
rpn_loc_loss =  0.011734861880540848 rpn_cls_loss =  0.23380979895591736 roi_loc_loss =  0.008787475526332855 roi_cls_loss =  0.8353948593139648
step 120
rpn_loc_loss =  0.009550822898745537 rpn_cls_loss =  0.2211441844701767 roi_loc_loss =  0.00949789397418499 roi_cls_loss =  0.7904484868049622
step 140
rpn_loc_loss =  0.06462231278419495 rpn_cls_loss =  0.33072441816329956 roi_loc_loss =  0.027402769774198532 roi_cls_loss =  1.1045066118240356
step 160
rpn_loc_loss =  0.026206055656075478 rpn_cls_loss =  0.11946636438369751 roi_loc_loss =  0.007503715343773365 roi_cls_loss =  0.35267403721809387
step 180
rpn_loc_loss =  0.03254791721701622 rpn_cls_loss =  0.15054623782634735 roi_loc

step 1140
rpn_loc_loss =  1.5023176670074463 rpn_cls_loss =  1.3525145053863525 roi_loc_loss =  0.026502493768930435 roi_cls_loss =  1.0618194341659546
step 1160
rpn_loc_loss =  0.1460960954427719 rpn_cls_loss =  0.7056381702423096 roi_loc_loss =  0.02487459033727646 roi_cls_loss =  0.9162350296974182
step 1180
rpn_loc_loss =  0.03248582035303116 rpn_cls_loss =  0.1577916294336319 roi_loc_loss =  0.02851036936044693 roi_cls_loss =  1.0601673126220703
step 1200
rpn_loc_loss =  0.07461521774530411 rpn_cls_loss =  0.26086699962615967 roi_loc_loss =  0.03105759061872959 roi_cls_loss =  1.2447465658187866
step 1220
rpn_loc_loss =  0.04428570717573166 rpn_cls_loss =  0.09236672520637512 roi_loc_loss =  0.027545247226953506 roi_cls_loss =  0.6256650686264038
step 1240
rpn_loc_loss =  0.04240615665912628 rpn_cls_loss =  0.32253435254096985 roi_loc_loss =  0.020342200994491577 roi_cls_loss =  1.2118980884552002
step 1260
rpn_loc_loss =  0.6709930896759033 rpn_cls_loss =  0.7263270616531372 roi_

step 2220
rpn_loc_loss =  0.19705894589424133 rpn_cls_loss =  0.6355140209197998 roi_loc_loss =  0.023129213601350784 roi_cls_loss =  1.2597156763076782
